In [18]:
import yfinance as yf
import boto
from os import environ as env

In [44]:

def calc_stock(high, current):
    ratio = (current - high)/high
    return ratio


def convert_tuple(tup): 
    string_tup =  "{} : {}".format(tup[0], str(tup[1]))
    return string_tup


def create_message(pairs):
    message = "Ordered Portfolio:\n"
    for pair in pairs:
        message += convert_tuple(pair) + "\n"
    return message
    
    
def publish_message_sns(message):
    sns_arn = env.get('SNS_ARN').strip()
    sns_client = boto3.client('sns')
    try:
        print(message)

        response = sns_client.publish(
            TopicArn=sns_arn,
            Message=message
        )

        print(response)

    except Exception as e:
        print("ERROR PUBLISHING MESSAGE TO SNS: {}".format(e))


def read_tickers(file='deployment/stock_tickers.txt'):
      
    pairs = dict()

    with open(file, 'r') as f:
        while True:
            
            try:
                # Get next line from file 
                ticker = (f.readline()).strip()
            
                if ticker == "":
                    break
                else:
                    print(ticker)

                stock = yf.Ticker(ticker)
                data = stock.history('5y')
                data.dropna()

                close = data.Close[-1]
                print(close)
                high = max(data.Close)

                delta = calc_stock(high, close)

                pairs[ticker] = delta

                if not ticker:
                    break
            except Exception as e:
                print(e)
                if not ticker:
                    break

    return(sorted(pairs.items(), key=lambda x: x[1]))


def handler():
    pairs = read_tickers()
    message = create_message(pairs)
    print(message)
    #AWS LIVE VERSION ONLY:
    #publish_message_sns(message)
    

In [45]:
handler()

ABT
             Open   High    Low  Close    Volume  Dividends  Stock Splits
Date                                                                     
2020-04-20  94.82  100.0  94.65   98.0  17221500        0.0             0
98.0
98.0
89.14
ABBV
             Open   High    Low  Close   Volume  Dividends  Stock Splits
Date                                                                    
2020-04-20  83.54  85.33  82.63  83.99  9125000        0.0             0
83.99
83.99
82.13
ADBE
              Open   High     Low   Close   Volume  Dividends  Stock Splits
Date                                                                       
2020-04-20  340.79  348.5  338.75  344.88  2771400          0             0
344.88
344.88
340.77
BABA
              Open   High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                        
2020-04-20  209.87  216.1  209.36  212.13  19272100          0             0
212.13
212.13
204.78
AGN
 

            Open   High    Low  Close     Volume  Dividends  Stock Splits
Date                                                                     
2020-04-20  94.7  97.24  93.98  94.58  5269300.0        0.0             0
94.58
94.58
98.5
MSFT
              Open    High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                         
2020-04-20  176.63  178.75  174.99  175.06  36595200        0.0             0
175.06
175.06
173.7
NFLX
              Open    High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                         
2020-04-20  435.17  444.49  430.56  437.49  12588000          0           0.0
437.49
437.49
413.55
NKE
             Open   High    Low  Close   Volume  Dividends  Stock Splits
Date                                                                    
2020-04-20  88.43  89.44  87.52   87.9  7243600        0.0           0.0
87.9
87.9
87.47
NOW